In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy

In [3]:
# Connect to a local SQLite database. SQLite is a simple file oriented database management "library." 
# in the following we launch a SQLite database stored in the .sqlite file.
dbfile = Path("ZMainBldg.sqlite")
#if dbfile.exists():
    #dbfile.unlink()

sqlite_uri = "sqlite:///ZMainBldg.sqlite"
sqlite_engine = sqlalchemy.create_engine(sqlite_uri)
sqlite_engine.

AttributeError: 'function' object has no attribute 'text_factory'

In [ ]:
# CREATE TABLE alameda AS SELECT * FROM main_bldg WHERE County='ALAMEDA';
sqlite_uri = "sqlite:///ZCountyExtracts.sqlite"
zce_engine = sqlalchemy.create_engine(sqlite_uri)

In [ ]:
# Check tables
sqlite_engine.table_names()

In [ ]:
sql_expr = """
SELECT * FROM main_bldg WHERE County = 'Los Angeles' LIMIT 2;
"""

alameda = pd.read_sql(sql_expr, sqlite_engine)
alameda.head()

In [ ]:
# Work on sample table for now. sample contains a Simple Random Sample of main_bldg of size 100,000
sql_expr = """
SELECT * FROM sample
GROUP BY ImportParcelID;
"""
sample = pd.read_sql(sql_expr, sqlite_engine)
sample.head()

In [ ]:
# 100000
len(sample)

In [ ]:
# Number of sample Alameda records
len(sample[sample['County'] == 'ALAMEDA'])

In [ ]:
# Get cities from JC RC Inventory
cities = list(pd.read_excel('jc_rc.xlsx')['Unnamed: 0'].values[1:-1])
cities

In [ ]:
zce_engine.table_names()

In [ ]:
alameda = alameda.groupby('ImportParcelID', as_index=False).first()

In [ ]:
# Number of Alameda records
len(alameda)

In [ ]:
# All the cities
alameda['PropertyCity'].value_counts()

In [ ]:
# New length
len(alameda)

In [ ]:
alameda['PropertyCity'].value_counts()

In [ ]:
# Load CHPC which will help determine if property is subsidized
chpc = pd.read_excel('chpc.xls', sheet_name='Cleaned')

In [ ]:
chpc.head(1)

In [ ]:
# Compare Address numbers and name 
chpc['Address_Num'] = chpc['Address_Cleaned'].str.extract(r'(\d+)')
chpc['Address_Name'] = chpc['Address_Cleaned'].str.extract(r'([^\d,-]\w+)').str.upper()
chpc.head(1)

In [ ]:
# Merge based on euclidean distance
chpc = chpc.dropna(subset=['Longitude', 'Latitude'])
def merge_euclidean(table, eps=0.01):
    chpc_list = []
    table['merge_row'] = table.index.values
    for i, row in table.iterrows():
        chpc_subset = chpc.loc[((chpc.Longitude - row.PropertyAddressLongitude)**2 + (chpc.Latitude - row.PropertyAddressLatitude)**2 < eps) & (chpc.Address_Num == row.PropertyHouseNumber)]
        chpc_subset['merge_row'] = i
        chpc_list.append(chpc_subset)
    chpc_found = pd.concat(chpc_list)

    result = pd.merge(table, chpc_found, on='merge_row', how='left')
    return result

In [ ]:
# Fix dtype for merging
sample.to_csv('sample.csv', sep='|', index=False)
sample = pd.read_csv('sample.csv', sep='|')
alameda.to_csv('alameda.csv', sep='|', index=False)
alameda = pd.read_csv('alameda.csv', sep='|')

In [ ]:
alameda.head(1)

In [ ]:
chpc_alameda = merge_euclidean(alameda, eps=0.0000001)

In [ ]:
len(chpc_alameda)

In [ ]:
chpc_alameda = (chpc_alameda.dropna(subset=['Longitude', 'Latitude']))

In [ ]:
display(chpc_alameda.head())
print(len(chpc_alameda))

In [ ]:
# Going to add subsidez column now
chpc_alameda = chpc_alameda[['ImportParcelID']]
chpc_alameda['Subsidized'] = 'Y'

In [ ]:
alameda = pd.merge(alameda, chpc_alameda, on=['ImportParcelID'], how='left')

In [ ]:
value = {'Subsidized': 'N'}
alameda.fillna(value=value, inplace=True)
alameda.head()

In [ ]:
alameda.to_csv('alameda_v2.csv', sep='|', index=False)

In [ ]:
len(alameda[alameda['Subsidized']=='Y'])

In [ ]:
a